# MongoDB with Python

In [ ]:
import pymongo
from bson.son import SON
from pprint import pprint

In [ ]:
client = pymongo.MongoClient('localhost', 27017)
db = client.db

## 1. CRUD

### Insert

In [ ]:
# Insert one item
db.inventory.insert_one(
    {"item": "canvas",
     "qty": 100,
     "status": "C",
     "tags": ["cotton"],
     "size": {"h": 28, "w": 35.5, "uom": "cm"},
     "dim_cm": [28, 35.5]})

In [ ]:
# 전부 삭제
db.inventory.delete_many({})

# Insert many items
db.inventory.insert_many([
   # MongoDB adds the _id field with an ObjectId if _id is not present
   { 
       "item": "journal", 
       "qty": 25, 
       "status": "A",
       "size": { 
           "h": 14, 
           "w": 21, 
           "uom": "cm" 
       }, 
       "tags": [ "blank", "red" ],
       "dim_cm": [14, 21]
   },
   { 
       "item": "notebook", 
       "qty": 50, 
       "status": "A",
       "size": { 
           "h": 8.5, 
           "w": 11, 
           "uom": "in" 
       }, 
       "tags": [ "red", "blank" ],
       "dim_cm": [14, 21]
   },
   { 
       "item": "paper", 
       "qty": 100, 
       "status": "D",
       "size": { 
           "h": 8.5, 
           "w": 11, 
           "uom": "in" 
       }, 
       "tags": [ "red", "blank", "plain" ],
       "dim_cm": [8.5, 11]
   },
   { 
       "item": "planner", 
       "qty": 75, 
       "status": "D",
       "size": { 
           "h": 22.85, 
           "w": 30, 
           "uom": "cm" 
       }, 
       "tags": [ "blank", "red" ] ,
       "dim_cm": [22.85, 30]
   },
   { 
       "item": "postcard", 
       "qty": 45, 
       "status": "A",
       "size": 
       { 
           "h": 10, 
           "w": 15.25, 
           "uom": "cm" 
       }, 
       "tags": [ "blue" ],
       "dim_cm": [10, 15.25]
   }
])

### Query

In [ ]:
# Query all documents
for item in db.inventory.find({}):
    print(item['item'])

In [ ]:
# Query exact matchs with dictionary
for item in db.inventory.find({'size' : { "h": 10, "w": 15.25, "uom": "cm" }}):
    print(item['item'], item['size'])

In [ ]:
# Query subdocument items
for item in db.inventory.find({"size.uom": "in"}):
    print(item['item'], item['size']['uom'])

In [ ]:
# SELECT * FROM inventory WHERE size.h
for item in db.inventory.find({'size.h': {'$gt' : 10 }}):
    print(item['item'], item['size']['h'])

In [ ]:
# SELECT * FROM inventory WHERE status in ("A", "D")
for item in db.inventory.find({'status': {"$in" : ['A', 'D']}}):
    print(item['item'], item['status'])

In [ ]:
# SELECT * FROM inventory WHERE status = 'D' AND qty > 50
for item in db.inventory.find({'status' : 'D', 'qty' : {'$gt' : 50 }}):
    print(item['item'], item['status'], item['qty'])

In [ ]:
# SELECT * FROM inventory WHERE status = 'C' AND qty < 50
for item in db.inventory.find({'$or' : [{'status' : 'D'}, {'qty': {'$lt' : 50}}]}):
    print(item['status'], item['qty'])

In [ ]:
# SELECT * FROM inventory WHERE status = "A" AND ( qty < 30 OR item LIKE "p%")
cursor = db.inventory.find({
    "status": "A",
    "$or": [{"qty": {"$lt": 30}}, {"item": {"$regex": "^p"}}]})

for item in cursor:
    print(item['status'], item['qty'], item['item'])

In [ ]:
# Exact maching
for item in db.inventory.find({"tags": ["red", "blank"]}):
    print(item['item'], item["tags"])

In [ ]:
# Query arrays which contains "red" and "black" in any order
for item in db.inventory.find({"tags": { '$all' : ["red", "blank"]}}):
    print(item['item'], item["tags"])

In [ ]:
# Query arrays which contains "red"
for item in db.inventory.find({"tags": "red"}):
    print(item['item'], item["tags"])

In [ ]:
#  Queries for all documents where the array dim_cm contains at least one element whose value is greater than 25.
for item in db.inventory.find({"dim_cm": {'$gt' : 25}}):
    print(item['item'], item["dim_cm"])

In [ ]:
# 15보다 작은 하나의 엘리먼트가 하나 이상, 20보다 큰 엘리먼트가 하나 이상 있어야 한다.
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])
print('------')    
for item in db.inventory.find({"dim_cm": {"$lt": 15, "$gt": 20}}):
    print(item['item'], item["dim_cm"])

In [ ]:
# 'Or' condition
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])

print('------')  

for item in db.inventory.find({'$or' : [{"dim_cm": {"$lt": 9}}, {"dim_cm": {"$gt": 20}}]}):
    print(item['item'], item["dim_cm"])

In [ ]:
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])

print('------') 

# dim_cm에 두 조건을 모두 만족하는 엘리먼트가 하나 이상 있어야한다.
for item in db.inventory.find({"dim_cm": {"$elemMatch": {"$gt": 22, "$lt": 30}}}):
    print(item['item'], item["dim_cm"])

In [ ]:
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])

print('------') 

# INDEX를 이용한 접근 방법
for item in db.inventory.find({'dim_cm.1': {"$gt": 25}}):
    print(item['item'], item['dim_cm'])

In [ ]:
# Array 길이
# $gt를 사용해서 2보다 큰 Array 모두 반환하는 그런 문법은 안되는 듯.
for item in db.inventory.find({'tags' : {'$size': 2}}):
    print(item['item'], item['tags'])

In [ ]:
# 1보다 큰 Array는 아래 처럼 사용하면 됨
for item in db.inventory.find({'$where' : 'this.tags.length > 1'}):
    print(item['item'], item['tags'])

In [ ]:
# 서브 document가 배열 형태로 있는 예제를 수행하기 위해 새로운 DB생성
db.inventory.delete_many({})

db.inventory.insert_many([
    {"item": "journal",
     "instock": [
         SON([("warehouse", "A"), ("qty", 5)]),
         SON([("warehouse", "C"), ("qty", 15)])]},
    {"item": "notebook",
     "instock": [
         SON([("warehouse", "C"), ("qty", 5)])]},
    {"item": "paper",
     "instock": [
         SON([("warehouse", "A"), ("qty", 60)]),
         SON([("warehouse", "B"), ("qty", 15)])]},
    {"item": "planner",
     "instock": [
         SON([("warehouse", "A"), ("qty", 40)]),
         SON([("warehouse", "B"), ("qty", 5)])]},
    {"item": "postcard",
     "instock": [
         SON([("warehouse", "B"), ("qty", 15)]),
         SON([("warehouse", "C"), ("qty", 35)])]}])

In [ ]:
# 리스트 서브 문서 중 하나의 문서라도 조건을 만족하면 반환

for item in db.inventory.find({"instock": SON([("warehouse", "A"), ("qty", 5)])}):
    print(item['item'], item['instock'])
    
print('필드 위치 바뀌면 안된다.')    

for item in db.inventory.find({"instock": SON([("qty", 5), ("warehouse", "A")])}):
    print(item['item'], item['instock'])

In [ ]:
for item in db.inventory.find({}):
    print(item['item'], item['instock'])

print('- 하나의 문서라도 less than equal에 걸리면 반환')

for item in db.inventory.find({'instock.qty': {"$lte": 5}}):
    print(item['item'], item['instock'])

In [ ]:
for item in db.inventory.find({}):
    print(item['item'], item['instock'])

print('-각 엘리먼트의 qty가 두 조건을 모두 만족해야 함')

for item in db.inventory.find({"instock": {"$elemMatch": {"qty": {"$gt": 10, "$lte": 20}}}}):
    print(item['item'], item['instock'])

print('-각 엘리먼트의 qty가 아래 한 조건만 맞는 엘리만트가 있으면 반환')
    
for item in db.inventory.find({"instock.qty": {"$lt": 10, "$lte": 30}}):
    print(item['item'], item['instock'])
    
    

In [ ]:
for item in db.inventory.find({"instock.qty": 5, "instock.warehouse": "A"}):
    print(item['item'], item['instock'])

In [ ]:
db.inventory.delete_many({})

db.inventory.insert_many([
    {
        "item": "journal",
        "status": "A",
        "size": {"h": 14, "w": 21, "uom": "cm"},     
        "instock": [{"warehouse": "A", "qty": 5}]
    },
    {
        "item": "notebook",
        "status": "A",
        "size": {"h": 8.5, "w": 11, "uom": "in"},
        "instock": [{"warehouse": "C", "qty": 5}]
    },
    {
        "item": "paper",
        "status": "D",
        "size": {"h": 8.5, "w": 11, "uom": "in"},
        "instock": [{"warehouse": "A", "qty": 60}]
    }, 
    {
        "item": "planner",
        "status": "D",
        "size": {"h": 22.85, "w": 30, "uom": "cm"},
        "instock": [{"warehouse": "A", "qty": 40}]
    },
    {
        "item": "postcard",
        "status": "A",
        "size": {"h": 10, "w": 15.25, "uom": "cm"},
        "instock": [
            {"warehouse": "B", "qty": 15},
            {"warehouse": "C", "qty": 35}
        ]
    }
])


In [ ]:
# SELECT * from inventory WHERE status = "A"

for item in db.inventory.find({"status": "A"}):
    pprint(item)


In [ ]:
# SELECT _id, item, status from inventory WHERE status = "A"
#  _id, item, status 만 반환한다.

for item in db.inventory.find({"status": "A"}, {"item": 1, "status": 1}):
    pprint(item)

# _id는 제외. _id만 예외적으로 include, exclude에 섞어서 쓸 수 있다. _id를 instock으로 변경하면 에러.
for item in db.inventory.find({"status": "A"}, {"item": 1, "status": 1, '_id' : 0}):
    pprint(item)

# _id, instock, status를 제외한 나머지 반환
for item in db.inventory.find({"status": "A"}, {'_id' : 0, 'instock': 0, 'status': 0}):
    pprint(item)


In [ ]:
# .문법으로 서브 문서 접근하는 방법

for item in db.inventory.find({"status": "A"}, {'_id': 0, 'item': 1, 'status': 1, 'size.uom': 1}):
    pprint(item)

In [ ]:
# .문법으로 서브 문서 배열 접근하는 방법

for item in db.inventory.find({"status": "A"}, {'_id': 0, 'instock.qty': 1}):
    pprint(item)

In [ ]:
for item in db.inventory.find({"status": "A"}, {'_id': 0, 'item': 1, 'instock': 1}):
    pprint(item)
# $slice로 마지막 엘리먼트를 가지고 온다.        
for item in db.inventory.find({"status": "A"}, {'_id': 0, 'item': 1, 'instock': {'$slice': -1}}):
    pprint(item)

### Update

In [ ]:
db.inventory.delete_many({})

db.inventory.insert_many([
    {"item": "canvas",
     "qty": 100,
     "size": {"h": 28, "w": 35.5, "uom": "cm"},
     "status": "A"},
    {"item": "journal",
     "qty": 25,
     "size": {"h": 14, "w": 21, "uom": "cm"},
     "status": "A"},
    {"item": "mat",
     "qty": 85,
     "size": {"h": 27.9, "w": 35.5, "uom": "cm"},
     "status": "A"},
    {"item": "mousepad",
     "qty": 25,
     "size": {"h": 19, "w": 22.85, "uom": "cm"},
     "status": "P"},
    {"item": "notebook",
     "qty": 50,
     "size": {"h": 8.5, "w": 11, "uom": "in"},
     "status": "P"},
    {"item": "paper",
     "qty": 100,
     "size": {"h": 8.5, "w": 11, "uom": "in"},
     "status": "D"},
    {"item": "planner",
     "qty": 75,
     "size": {"h": 22.85, "w": 30, "uom": "cm"},
     "status": "D"},
    {"item": "postcard",
     "qty": 45,
     "size": {"h": 10, "w": 15.25, "uom": "cm"},
     "status": "A"},
    {"item": "sketchbook",
     "qty": 80,
     "size": {"h": 14, "w": 21, "uom": "cm"},
     "status": "A"},
    {"item": "sketch pad",
     "qty": 95,
     "size": {"h": 22.85, "w": 30.5, "uom": "cm"},
     "status": "A"}
])

In [ ]:
# 단일 업데이트
# {"item": "paper"} 매칭 아이템을 $set으로 수정하고 "$currentDate" 현재 시간 항목 추가


db.inventory.update_one(
    {"item": "paper"},
    {"$set": {"size.uom": "cm", "status": "P"},
     "$currentDate": {"lastModified": True}})

print (db.inventory.find_one({"item": "paper"}))

In [ ]:
# 다수 항목 수정

db.inventory.update_many(
    {"qty": {"$lt": 50}},
    {"$set": {"size.uom": "in", "status": "P"},
     "$currentDate": {"lastModified": True}})

for item in db.inventory.find({"qty": {"$lt": 50}}):
    pprint(item)

In [ ]:
# _id를 제외하고 전체 문서 교체

db.inventory.replace_one(
    {"item": "paper"},
    {"item": "paper",
     "instock": [
         {"warehouse": "A", "qty": 60},
         {"warehouse": "B", "qty": 50}]}
)

print (db.inventory.find_one({"item": "paper"}))

### Delete

In [ ]:
# 삭제

db.inventory.delete_many({"status": "A"})
db.inventory.delete_one({"status": "D"})

### Bulk Write
There are two types of bulk write operations

* Ordered : 순차 실행, 실행 중 실패하면 즉각 반환
* Unordered : 병렬 처리: 실행 중 실패해도 나머지 실행

It supports the folloings.

* insertOne
* updateOne
* updateMany
* replaceOne
* deleteOne
* deleteMany

In [ ]:
db.schools.delete_many({})

db.schools.insert_many([
    { "_id" : 1, "char" : "Brisbane", "class" : "monk", "lvl" : 4 },
    { "_id" : 2, "char" : "Eldon", "class" : "alchemist", "lvl" : 3 },
    { "_id" : 3, "char" : "Meldane", "class" : "ranger", "lvl" : 3 }
])

In [ ]:
from pymongo import InsertOne, DeleteOne, UpdateOne, ReplaceOne
from pymongo.errors import BulkWriteError

operations = [
    InsertOne({"_id" : 4, "char" : "Dithras", "class" : "barbarian", "lvl" : 4}),
    InsertOne({"_id" : 5, "char" : "Taeln", "class" : "fighter", "lvl" : 3}),
    UpdateOne({"char" : "Eldon"}, { "$set" : { "status" : "Critical Injury" } }) ,
    DeleteOne({ "char" : "Brisbane"}),
    ReplaceOne({ "char" : "Meldane" }, { "char" : "Tanys", "class" : "oracle", "lvl" : 4 })
]
    
try:    
    db.schools.bulk_write(operations, ordered=True)
except BulkWriteError as bwe:
    pprint(bwe.details)
    

### Text Search

mongodb는 문장 검색 기능을 제공한다. 문장 검색 기능을 이용하기 위해서는 index 생성이 선행되어야 한다.

In [ ]:
from pymongo import IndexModel, TEXT

db.stores.delete_many({})

for index in db.stores.index_information():
    if index == 'text_index':
        db.stores.drop_index('text_index')
    
db.stores.insert_many(
   [
     { 'name': "Java Hut", 'description': "Coffee and cakes" },
     { 'name': "Burger Buns", 'description': "Gourmet hamburgers" },
     { 'name': "Coffee Shop", 'description': "Just coffee" },
     { 'name': "Clothes Clothes Clothes", 'description': "Discount clothing" },
     { 'name': "Java Shopping", 'description': "Indonesian goods" }
   ]
)

# text 검색을 위해서는 TEXT index가 설정되어야 한다.
db.stores.create_index([('name', pymongo.TEXT), ('description', pymongo.TEXT)], name='text_index')


In [ ]:
# java, shop, discount 개별 단어를 검색하기 위한 옵션
for item in db.stores.find({'$text':{'$search':"java shop discount"}}):
    print(item['name'], '-', item['description'])
    

In [ ]:
# java, shop 개별 단어를 검색, 하지만 coffee 단어는 제외
for item in db.stores.find({'$text':{'$search':"java shop -coffee"}}):
    print(item['name'], '-', item['description'])

In [ ]:
# "java shop" 전체 단어를 검색하기 위한 옵션
for item in db.stores.find({'$text':{'$search':"\"java shop\""}}):
    print(item['name'], '-', item['description'])
    

## 2. Indexes

In [ ]:
import pandas as pd
import json

data = pd.read_csv('movies_initial.csv')
payload = json.loads(data.to_json(orient='records'))
db.movie.delete_many({})
db.movie.insert_many(payload)


### Single field indexes

In [ ]:
for index in db.movie.index_information():
    if index == 'title_index':
        db.movie.drop_index('title_index')
        
without_index = db.movie.find({ 'title' : 'A Turn of the Century Illusionist'}).explain()['executionStats']['executionTimeMillis']
db.movie.create_index([('title', pymongo.ASCENDING)], name='title_index')
with_index = db.movie.find({ 'title' : 'A Turn of the Century Illusionist'}).explain()['executionStats']['executionTimeMillis']

print('executionTimeMillis without index', without_index)
print('executionTimeMillis with index', with_index)

### Compound indexes

In [ ]:
# Delelte index
for index in db.movie.index_information():
    if index == 'genre_index':
        db.movie.drop_index('genre_index')

without_index = db.movie.find(
    { 
        'genre' : 'Documentary, Short', 
        'imdbVotes': {'$gt' : 1000}
    }).explain()['executionStats']['executionTimeMillis']

print('executionTimeMillis without index', without_index)

# Create Single index
db.movie.create_index([('genre', pymongo.ASCENDING)], name='genre_index')

with_single_index = db.movie.find(
    { 
        'genre' : 'Documentary, Short', 
        'imdbVotes': {'$gt' : 1000}
    }).explain()['executionStats']['executionTimeMillis']

print('executionTimeMillis with single index', with_single_index)

# Delete Single index
for index in db.movie.index_information():
    if index == 'genre_index':
        db.movie.drop_index('genre_index')

# Create Compound Index
db.movie.create_index([
    ('genre', pymongo.ASCENDING), 
    ('imdbVotes', pymongo.ASCENDING)], 
    name='genre_index')

with_compound_index = db.movie.find(
    { 
        'genre' : 'Documentary, Short', 
        'imdbVotes': {'$gt' : 1000}
    }).explain()['executionStats']['executionTimeMillis']

print('executionTimeMillis with compound index', with_compound_index)

## 3. Aggregation
